# CS246 - Colab 1
## Word Count in Spark

### Setup

Let's set up Spark on your Colab environment.  Run the cell below!

In [42]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

openjdk-8-jdk-headless is already the newest version (8u382-ga-1~22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.


Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [43]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [44]:
id='1SE6k_0YukzGd5wK-E4i6mG83nydlfvSa'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('pg100.txt')

If you executed the cells above, you should be able to see the file *pg100.txt* under the "Files" tab on the left panel.

### Your task

If you successfully run the setup stage, you are ready to work on the *pg100.txt* file which contains a copy of the complete works of Shakespeare.

Write a Spark application which outputs the number of words that start with each letter. This means that for every letter, we want to count the total number of (non-unique) words that start with a specific letter.

In your implementation, **ignore the letter case**, i.e., consider all words as lower case. Also, you can ignore all words that **start** with a non-alphabetic character. You should output word counts for the **entire document**, inclusive of the title, author, and the main texts. If you encounter words broken as a result of new lines, e.g. "pro-ject" where the segment after the dash sign is on a new line, no special processing is needed and you can safely consider it as two words.

Your outputs will be graded on a range -- if your differences from the ground-truths are within an error threshold of 5, you'll be considered correct.

In [45]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd
import re
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [46]:
# YOUR
txt = spark.read.text("pg100.txt")

text = txt.rdd.map(lambda x: x[0])

text.collect()

words = text.map(lambda word: word.lower()).flatMap(lambda line: line.split(" ")).filter(lambda word: re.match(r'^[a-z]+$', word))

letter_count = words.flatMap(lambda word: [(letter, 1) for letter in word])

word_counts = letter_count.reduceByKey(lambda a, b: a + b)

for (letter, count) in word_counts.collect():
    print(f"{letter}: {count}")

spark.stop()

h: 183474
p: 34506
r: 147391
j: 2368
c: 50827
g: 42358
b: 42527
l: 103136
s: 155537
i: 173557
y: 69844
d: 97446
t: 246677
e: 301329
o: 221677
u: 88725
n: 164582
k: 23062
f: 59507
m: 71959
w: 65031
a: 194986
v: 25924
x: 3451
q: 1983
z: 633


In [47]:
# CODE
spark.stop()

In [48]:
# HERE

Once you obtained the desired results, **head over to Gradescope and submit your solution for this Colab**!